In [7]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo
import pandas as pd

# Step 1: Fetch a list of active players
active_players_list = players.get_active_players()
first_100_active_players = active_players_list[:10]  # Select the first 100 players

# Step 2: Fetch detailed information for each of the first 100 active players
def get_player_info(player_id):
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
    return player_info.get_data_frames()[0]

# Store all player data in a DataFrame
all_players_data = pd.DataFrame()

for player in first_100_active_players:
    player_data = get_player_info(player['id'])
    all_players_data = pd.concat([all_players_data, player_data], ignore_index=True)

# Now all_players_data contains the detailed info for the first 100 active players
print(all_players_data[['PERSON_ID', 'DISPLAY_FIRST_LAST', 'POSITION', 'TEAM_NAME']])


Pandas imported successfully in nba_api, version: 2.2.2


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443
DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/commonplayerinfo?LeagueID=&PlayerID=1630173 HTTP/1.1" 200 654
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


Headers (simple): ['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST', 'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG', 'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT', 'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS', 'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE', 'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG']
Headers (simple): ['PLAYER_ID', 'PLAYER_NAME', 'TimeFrame', 'PTS', 'AST', 'REB', 'PIE']
Headers (simple): ['SEASON_ID']


DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/commonplayerinfo?LeagueID=&PlayerID=203500 HTTP/1.1" 200 697
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


Headers (simple): ['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST', 'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG', 'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT', 'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS', 'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE', 'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG']
Headers (simple): ['PLAYER_ID', 'PLAYER_NAME', 'TimeFrame', 'PTS', 'AST', 'REB', 'PIE']
Headers (simple): ['SEASON_ID']


DEBUG:urllib3.connectionpool:https://stats.nba.com:443 "GET /stats/commonplayerinfo?LeagueID=&PlayerID=1628389 HTTP/1.1" 200 686
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): stats.nba.com:443


Headers (simple): ['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST', 'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG', 'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT', 'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS', 'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE', 'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG']
Headers (simple): ['PLAYER_ID', 'PLAYER_NAME', 'TimeFrame', 'PTS', 'AST', 'REB', 'PIE']
Headers (simple): ['SEASON_ID']


KeyboardInterrupt: 

In [31]:
def scrape_team_roster(page, url):
    page.get(url)
    time.sleep(2)  # Let the page load

    # Find the roster table and process each row
    team_roster = []
    rows = page.elems('#roster tr')[1:]  # Skip the header row

    for row in rows:
        cols = row.elems('td')
        if cols:
            player_data = {
                'No.': cols[0].text,
                'Player': cols[1].text,
                'Pos': cols[2].text,
                'Ht': cols[3].text,
                'Wt': cols[4].text,
                'Birth Date': cols[5].text,
                'Exp': cols[6].text,
                'College': cols[7].text if len(cols) > 7 else 'N/A'
            }
            team_roster.append(player_data)
    return team_roster

def get_teams_and_scrape(page):
    base_url = 'https://fantasydata.com/nba/fantasy-basketball-leaders?scope=1&season=2024&seasontype=1&team=3&conference=1&date=05-02-2024'
    page.get(f"{base_url}")
    time.sleep(2)  # Ensure the page has loaded

   

    all_teams_data = []

    for season in ['2023', '2024']:
        for link in teams_links:
            team_url = f"{base_url}{link.attr('href')}"
            team_url = team_url.replace('.html', f'/{season}.html')
            print(f"Scraping {team_url}")

            # Check if the team abbreviation is "BOS" (Boston Celtics)
            if "BOS" in team_url:
                team_roster = scrape_team_roster(page, team_url)
                all_teams_data.append({'team': link.text, 'season': season, 'roster': team_roster})

    return all_teams_data


In [32]:
# Using ChromiumPage to control the browser
page = ChromiumPage()

# Gather data
nba_data = get_teams_and_scrape(page)

# Output the data or process as needed
for team_data in nba_data:
    print(team_data)

# Always remember to close the session when done
page.close()


No teams found with the given selector. Check if the correct method is used.


In [44]:
from DrissionPage import ChromiumPage
import time
import pandas as pd
from bs4 import BeautifulSoup

def scrape_team_roster(page, team_name, season):
    url = f'https://www.basketball-reference.com/teams/{team_name}/{season}.html'
    page.get(url)
    time.sleep(2)  # Let the page load
    
    # Get the HTML content of the roster table
    roster_table = page.ele('#roster').html
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(roster_table, 'html.parser')
    
    # Find all rows in the table body
    rows = soup.select('tbody tr')
    
    # Initialize an empty list to store player data
    data = []

    # Process each row and extract player data
    for row in rows:
        cols = row.find_all('td')
        player_data = {
            'No.': cols[0].text,
            'Player': cols[1].text,
            'Pos': cols[2].text,
            'Ht': cols[3].text,
            'Wt': cols[4].text,
            'Birth Date': cols[5].text,
            'Exp': cols[6].text,
            'College': cols[7].text if len(cols) > 7 else 'N/A',
            'Season': season
        }
        data.append(player_data)
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    
    return df

# Using ChromiumPage to control the browser
page = ChromiumPage()

# List of team names for all NBA leagues
team_names = [
    'ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 
    'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 
    'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS'
]

# List of seasons
seasons = ['2023', '2024']

# Initialize an empty list to store DataFrame for each team
all_team_rosters = []

# Loop through each season
for season in seasons:
    # Loop through each team name
    for team_name in team_names:
        # Scrape the roster data for the team and season
        team_roster_df = scrape_team_roster(page, team_name, season)
        
        # Append the DataFrame to the list
        all_team_rosters.append(team_roster_df)

# Concatenate all DataFrames into one
consolidated_df = pd.concat(all_team_rosters, ignore_index=True)

# Print the consolidated DataFrame
print(consolidated_df)

# Always remember to close the session when done
page.close()


                          No. Player   Pos   Ht                  Wt  \
0                  Saddiq Bey     SF   6-7  215       April 9, 1999   
1           Bogdan Bogdanović     SG   6-5  220     August 18, 1992   
2                Clint Capela      C  6-10  240        May 18, 1994   
3                John Collins     PF   6-9  235  September 23, 1997   
4              Jarrett Culver     SF   6-6  195   February 20, 1999   
...                       ...    ...   ...  ...                 ...   
1139        Marvin Bagley III      C  6-10  235      March 14, 1999   
1140      Jules Bernard  (TW)     SG   6-7  210    January 21, 2000   
1141           Richaun Holmes     PF  6-10  235    October 15, 1993   
1142  Justin Champagnie  (TW)     SF   6-6  200       June 29, 2001   
1143         Tristan Vukcevic      C   7-0  223      March 11, 2003   

     Birth Date Exp        College Season  
0            us   2      Villanova   2023  
1            rs   5                  2023  
2            ch

In [45]:
# Export the consolidated DataFrame to a CSV file
consolidated_df.to_csv('team_rosters.csv', index=False)


In [7]:
from DrissionPage import ChromiumPage
import time
import pandas as pd
from bs4 import BeautifulSoup

def scrape_team_roster(page, team_name):
    url = f'https://www.basketball-reference.com/teams/{team_name}/2024.html'
    page.get(url)
    time.sleep(2)  # Let the page load
    
    # Get the HTML content of the roster table
    roster_table = page.ele('#roster').html
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(roster_table, 'html.parser')
    
    # Find all rows in the table body
    rows = soup.select('tbody tr')
    
    # Initialize an empty list to store player data
    data = []

    # Process each row and extract player data
    for row in rows:
        cols = row.find_all('td')
        player_data = {
            'Player': cols[0].text,
            'Pos': cols[1].text,
            'Ht': cols[2].text,
            'Wt': cols[3].text,
            'Birth Date': cols[4].text,
            'Country': cols[5].text,
            'Exp': cols[6].text,
            'College': cols[7].text if len(cols) > 7 else 'N/A'
        }
        data.append(player_data)
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    
    return df

# Using ChromiumPage to control the browser
page = ChromiumPage()

# List of team names for all NBA leagues
team_names = [
    'ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 
    'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 
    'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS'
]

# Initialize an empty list to store DataFrame for each team
all_team_rosters = []

# Loop through each team name
for team_name in team_names:
    # Scrape the roster data for the team
    team_roster_df = scrape_team_roster(page, team_name)
    
    # Add team name as a column in the DataFrame
    team_roster_df['Team'] = team_name
    
    # Append the DataFrame to the list
    all_team_rosters.append(team_roster_df)

# Concatenate all DataFrames into one
consolidated_df = pd.concat(all_team_rosters, ignore_index=True)

# Print the consolidated DataFrame
print(consolidated_df)

# Always remember to close the session when done
page.close()


                      Player Pos    Ht   Wt          Birth Date Country Exp  \
0          Bogdan Bogdanović  SG   6-5  220     August 18, 1992      rs   6   
1            Dejounte Murray  SG   6-5  180  September 19, 1996      us   6   
2               Clint Capela   C  6-10  240        May 18, 1994      ch   9   
3           Garrison Mathews  SG   6-5  215    October 24, 1996      us   4   
4                 Saddiq Bey  SF   6-7  215       April 9, 1999      us   3   
..                       ...  ..   ...  ...                 ...     ...  ..   
527        Marvin Bagley III   C  6-10  235      March 14, 1999      us   5   
528      Jules Bernard  (TW)  SG   6-7  210    January 21, 2000      us   R   
529           Richaun Holmes  PF  6-10  235    October 15, 1993      us   8   
530  Justin Champagnie  (TW)  SF   6-6  200       June 29, 2001      us   2   
531         Tristan Vukcevic   C   7-0  223      March 11, 2003      it   R   

           College Team  
0                   ATL  

In [9]:
# Remove (TW) from player names
consolidated_df['Player'] = consolidated_df['Player'].str.replace(r'\s*\(TW\)', '', regex=True)


In [14]:
consolidated_df

,Player,Pos,Ht,Wt,Birth Date,Country,Exp,College,Team
0,Bogdan Bogdanović,SG,6-5,220,"August 18, 1992",rs,6,,ATL
1,Dejounte Murray,SG,6-5,180,"September 19, 1996",us,6,Washington,ATL
2,Clint Capela,C,6-10,240,"May 18, 1994",ch,9,,ATL
3,Garrison Mathews,SG,6-5,215,"October 24, 1996",us,4,Lipscomb,ATL
4,Saddiq Bey,SF,6-7,215,"April 9, 1999",us,3,Villanova,ATL
...,...,...,...,...,...,...,...,...,...
527,Marvin Bagley III,C,6-10,235,"March 14, 1999",us,5,Duke,WAS
528,Jules Bernard,SG,6-7,210,"January 21, 2000",us,R,UCLA,WAS
529,Richaun Holmes,PF,6-10,235,"October 15, 1993",us,8,Bowling Green,WAS
530,Justin Champagnie,SF,6-6,200,"June 29, 2001",us,2,Pitt,WAS


In [12]:
# Export the consolidated DataFrame to a CSV file with specified encoding
consolidated_df.to_csv('team_rosters_2024.csv', index=False, encoding='utf-8-sig')



In [1]:
from DrissionPage import ChromiumPage
import time
import pandas as pd
from bs4 import BeautifulSoup

def scrape_team_roster(page, team_name):
    url = f'https://www.basketball-reference.com/teams/{team_name}/2023.html'
    page.get(url)
    time.sleep(2)  # Let the page load
    
    # Get the HTML content of the roster table
    roster_table = page.ele('#roster').html
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(roster_table, 'html.parser')
    
    # Find all rows in the table body
    rows = soup.select('tbody tr')
    
    # Initialize an empty list to store player data
    data = []

    # Process each row and extract player data
    for row in rows:
        cols = row.find_all('td')
        player_data = {
            'Player': cols[0].text,
            'Pos': cols[1].text,
            'Ht': cols[2].text,
            'Wt': cols[3].text,
            'Birth Date': cols[4].text,
            'Country': cols[5].text,
            'Exp': cols[6].text,
            'College': cols[7].text if len(cols) > 7 else 'N/A'
        }
        data.append(player_data)
    
    # Convert the list of dictionaries to a DataFrame
    df = pd.DataFrame(data)
    
    return df

# Using ChromiumPage to control the browser
page = ChromiumPage()

# List of team names for all NBA leagues
team_names = [
    'ATL', 'BOS', 'BRK', 'CHO', 'CHI', 'CLE', 'DAL', 'DEN', 'DET', 'GSW', 
    'HOU', 'IND', 'LAC', 'LAL', 'MEM', 'MIA', 'MIL', 'MIN', 'NOP', 'NYK', 
    'OKC', 'ORL', 'PHI', 'PHO', 'POR', 'SAC', 'SAS', 'TOR', 'UTA', 'WAS'
]

# Initialize an empty list to store DataFrame for each team
all_team_rosters = []

# Loop through each team name
for team_name in team_names:
    # Scrape the roster data for the team
    team_roster_df = scrape_team_roster(page, team_name)
    
    # Add team name as a column in the DataFrame
    team_roster_df['Team'] = team_name
    
    # Append the DataFrame to the list
    all_team_rosters.append(team_roster_df)

# Concatenate all DataFrames into one
consolidated_df_2023 = pd.concat(all_team_rosters, ignore_index=True)

# Print the consolidated DataFrame
print(consolidated_df_2023)

# Always remember to close the session when done
page.close()


                 Player Pos    Ht   Wt          Birth Date Country Exp  \
0            Saddiq Bey  SF   6-7  215       April 9, 1999      us   2   
1     Bogdan Bogdanović  SG   6-5  220     August 18, 1992      rs   5   
2          Clint Capela   C  6-10  240        May 18, 1994      ch   8   
3          John Collins  PF   6-9  235  September 23, 1997      us   5   
4        Jarrett Culver  SF   6-6  195   February 20, 1999      us   3   
..                  ...  ..   ...  ...                 ...     ...  ..   
607       Kendrick Nunn  SG   6-2  190      August 3, 1995      us   2   
608  Kristaps Porziņģis   C   7-2  240      August 2, 1995      lv   6   
609      Jordan Schakel  SF   6-6  200       June 13, 1998      us   1   
610         Isaiah Todd  PF  6-10  220    October 17, 2001      us   1   
611        Delon Wright  PG   6-5  185      April 26, 1992      us   7   

               College Team  
0            Villanova  ATL  
1                       ATL  
2                    

In [3]:
# Remove (TW) from player names
consolidated_df_2023['Player'] = consolidated_df_2023['Player'].str.replace(r'\s*\(TW\)', '', regex=True)
# Export the consolidated DataFrame to a CSV file with specified encoding
consolidated_df_2023.to_csv('team_rosters_2023.csv', index=False, encoding='utf-8-sig')



In [4]:
import pandas as pd
from unidecode import unidecode

# Function to transliterate names
def transliterate_name(name):
    return unidecode(name)

# Load the Excel file (make sure to specify the correct path and file name)
df = pd.read_csv('team_rosters_2023.csv')  # Adjust this line if your file name or path is different

# Apply transliteration to the relevant column (assuming the column with names is called 'PlayerName')
df['Player'] = df['Player'].apply(transliterate_name)

# Export the DataFrame to a CSV file with specified encoding
df.to_csv('team_rosters_2023_converted.csv', index=False, encoding='utf-8-sig')

print("Data has been successfully converted and saved.")


Data has been successfully converted and saved.
